In [ ]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [ ]:
df = pd.read_csv('/content/algo.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
def distance(loc1,loc2):
    lat1=loc1[0]
    lon1=loc1[1]
    lat2=loc2[0]
    lon2=loc2[1]
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    c = 2 * asin(sqrt(a))

    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371

    # calculate the result
    return(c * r)

In [ ]:
def nearestCluster(startPos,allPos):
  currShortestDistance=1000001
  currShortestIndex=0
  i=0
  for pos in allPos:
    tmpDis = distance(startPos,pos)
    if tmpDis == 0:
      tmpDis = 1000000
    if (tmpDis<currShortestDistance):
      currShortestDistance=tmpDis
      currShortestIndex=i
    i+=1
  return allPos[currShortestIndex]


In [ ]:
def nearestClusters(startPos,allPos,numOfNearest,distanceThreshold):
  currShortestDistances=[1000001]*numOfNearest
  currShortestIndices=[0]*numOfNearest

  i=0
  for pos in allPos:
    max_value = max(currShortestDistances)
    max_index = currShortestDistances.index(max_value)
    tmpDis = distance(startPos,pos)
    if tmpDis == 0:
      continue
    if (tmpDis<currShortestDistances[max_index]):
      currShortestDistances[max_index]=tmpDis
      currShortestIndices[max_index]=i
    i+=1
  res=[]
  passedThreshold=False
  while (len(currShortestDistances)>0 and not passedThreshold):
    tmpShortestDis=currShortestDistances[0]
    tmpShortestIndex=currShortestIndices[0]
    for i in range(0,len(currShortestDistances)):
      if(currShortestDistances[i]<tmpShortestDis):
        tmpShortestDis=currShortestDistances[i]
        tmpShortestIndex=currShortestIndices[i]
    if(tmpShortestDis<=distanceThreshold):
      res.append(allPos[tmpShortestIndex])
      currShortestDistances.remove(tmpShortestDis)
      currShortestIndices.remove(tmpShortestIndex)
    else:
      passedThreshold=True
      break
  return res


In [ ]:
startPos=[31.185606,31.525233]
allPos=[[29.41089313,30.86269592],[29.23869977,30.79355955],[29.36740351,30.68666862]]
nearestClusters(startPos,allPos,3,230)

In [ ]:
all_lats_longs = []
for i in range(len(df)):
    all_lats_longs.append(list(df[['Lat', 'Long']].iloc[i]))

In [ ]:
all_lats_longs

In [ ]:
for i in range(len(df)):
    nearest_cluster = nearestCluster(list(df[['Lat', 'Long']].iloc[i]),all_lats_longs)
    df.loc[i, 'nearest_lat'] = nearest_cluster[0]
    df.loc[i, 'nearest_long'] = nearest_cluster[1]

In [ ]:
def colorATMs(startPos,all_lats_longs):
  colors=["yellow"]*len(all_lats_longs)
  startIndex=all_lats_longs.index(startPos)
  colors[startIndex]='blue'
  nearestIndex= all_lats_longs.index(nearestCluster(startPos,all_lats_longs))
  colors[nearestIndex]='green'
  tmpDF=pd.DataFrame(all_lats_longs)
  tmpDF['color']=colors
  tmpDF.rename(columns = {0:'lat'}, inplace = True)
  tmpDF.rename(columns = {1:'long'}, inplace = True)
  return tmpDF


In [ ]:
colorATMs([28.31248558880002, 30.71705917280002],all_lats_longs)

In [ ]:
df

In [ ]:
for i in range(len(df)):
    for j in range(len(df)):
        if list(df[['nearest_lat', 'nearest_long']].iloc[i]) == list(df[['Lat', 'Long']].iloc[j]):
            df.loc[i, 'nearest_region'] = df['Region'].iloc[j]

In [ ]:
df.head()

In [ ]:
df.to_csv('clusters_with_nearest.csv', index=False)